In [1]:
import os
import cv2
import numpy as np
import torch
from torchvision import transforms, models
from torch.utils.data import DataLoader
from skimage.feature import graycomatrix, graycoprops, hog
import pywt
from PIL import Image
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [2]:
# Define directories
data_dir = "/raid/home/posahemanth/Phase2/Data"
processed_dir = "./Processed_Data"
os.makedirs(processed_dir, exist_ok=True)
IMG_SIZE = (224, 224)  # Default size, adjusted for specific models below

In [3]:
# Load images function (unchanged)
def load_ct_images(dataset_path):
    data = []
    labels = []
    classes = os.listdir(dataset_path)
    for class_name in classes:
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_path):
            continue
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, IMG_SIZE)
                img = img / 255.0
                data.append(img)
                labels.append(class_name)
    return np.array(data), np.array(labels)

In [4]:
# Load dataset
train_images, train_labels = load_ct_images(os.path.join(data_dir, "train"))
valid_images, valid_labels = load_ct_images(os.path.join(data_dir, "valid"))
test_images, test_labels = load_ct_images(os.path.join(data_dir, "test"))

In [5]:
print(f"Train Images: {train_images.shape}, Train Labels: {len(train_labels)}")
print(f"Validation Images: {valid_images.shape}, Validation Labels: {len(valid_labels)}")
print(f"Test Images: {test_images.shape}, Test Labels: {len(test_labels)}")


Train Images: (613, 224, 224), Train Labels: 613
Validation Images: (72, 224, 224), Validation Labels: 72
Test Images: (315, 224, 224), Test Labels: 315


In [6]:
# Define transforms for each model
resnet_transform = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
inception_transform = transforms.Compose([
    transforms.Resize((299, 299)), transforms.ToTensor(),  # Inception V3 uses 299x299
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
efficient_transform = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
densenet_transform = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
molmo_transform = transforms.Compose([  # Placeholder, adjust if actual specs available
    transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
# Load pre-trained models and adjust for feature extraction
# ResNet50
resnet50_model = models.resnet50(pretrained=True)
resnet50_model = torch.nn.Sequential(*(list(resnet50_model.children())[:-1]))  # Remove fc layer
resnet50_model.eval()

# Inception V3 - Corrected feature extractor
inceptionv3_model = models.inception_v3(pretrained=True)
class InceptionFeatureExtractor(torch.nn.Module):
    def __init__(self, original_model):
        super(InceptionFeatureExtractor, self).__init__()
        self.conv2d_1a_3x3 = original_model.Conv2d_1a_3x3
        self.conv2d_2a_3x3 = original_model.Conv2d_2a_3x3
        self.conv2d_2b_3x3 = original_model.Conv2d_2b_3x3
        self.maxpool1 = original_model.maxpool1
        self.conv2d_3b_1x1 = original_model.Conv2d_3b_1x1
        self.conv2d_4a_3x3 = original_model.Conv2d_4a_3x3
        self.maxpool2 = original_model.maxpool2
        self.mixed_5b = original_model.Mixed_5b
        self.mixed_5c = original_model.Mixed_5c
        self.mixed_5d = original_model.Mixed_5d
        self.mixed_6a = original_model.Mixed_6a
        self.mixed_6b = original_model.Mixed_6b
        self.mixed_6c = original_model.Mixed_6c
        self.mixed_6d = original_model.Mixed_6d
        self.mixed_6e = original_model.Mixed_6e
        self.mixed_7a = original_model.Mixed_7a
        self.mixed_7b = original_model.Mixed_7b
        self.mixed_7c = original_model.Mixed_7c
        self.avgpool = torch.nn.AdaptiveAvgPool2d((1, 1))  # Pool to 1x1
    
    def forward(self, x):
        x = self.conv2d_1a_3x3(x)
        x = self.conv2d_2a_3x3(x)
        x = self.conv2d_2b_3x3(x)
        x = self.maxpool1(x)
        x = self.conv2d_3b_1x1(x)
        x = self.conv2d_4a_3x3(x)
        x = self.maxpool2(x)
        x = self.mixed_5b(x)
        x = self.mixed_5c(x)
        x = self.mixed_5d(x)
        x = self.mixed_6a(x)
        x = self.mixed_6b(x)
        x = self.mixed_6c(x)
        x = self.mixed_6d(x)
        x = self.mixed_6e(x)
        x = self.mixed_7a(x)
        x = self.mixed_7b(x)
        x = self.mixed_7c(x)
        x = self.avgpool(x)
        return x

inceptionv3_model = InceptionFeatureExtractor(inceptionv3_model)
inceptionv3_model.eval()

# EfficientNetB0
efficientnetb0_model = models.efficientnet_b0(pretrained=True)
efficientnetb0_model = torch.nn.Sequential(*(list(efficientnetb0_model.children())[:-1]))  # Remove classifier
efficientnetb0_model.eval()

# DenseNet121
densenet121_model = models.densenet121(pretrained=True)
densenet121_model = torch.nn.Sequential(*(list(densenet121_model.children())[:-1]))  # Remove classifier
densenet121_model.eval()

# Molmo 7B D (Placeholder)
molmo7bd_model = models.resnet50(pretrained=True)  # Temporary substitute
molmo7bd_model = torch.nn.Sequential(*(list(molmo7bd_model.children())[:-1]))
molmo7bd_model.eval()

/raid/home/posahemanth/miniconda3/envs/tf_gpu_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/raid/home/posahemanth/miniconda3/envs/tf_gpu_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/raid/home/posahemanth/miniconda3/envs/tf_gpu_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [8]:
# Feature extraction functions
def extract_resnet50_features(image_pil):
    image = resnet_transform(image_pil).unsqueeze(0)
    with torch.no_grad():
        features = resnet50_model(image)
    return features.squeeze().numpy().flatten()

def extract_inceptionv3_features(image_pil):
    image = inception_transform(image_pil).unsqueeze(0)
    with torch.no_grad():
        features = inceptionv3_model(image)
    return features.squeeze().numpy().flatten()

def extract_efficientnetb0_features(image_pil):
    image = efficient_transform(image_pil).unsqueeze(0)
    with torch.no_grad():
        features = efficientnetb0_model(image)
    return features.squeeze().numpy().flatten()

def extract_densenet121_features(image_pil):
    image = densenet_transform(image_pil).unsqueeze(0)
    with torch.no_grad():
        features = densenet121_model(image)
    return features.squeeze().numpy().flatten()

def extract_molmo7bd_features(image_pil):  # Placeholder
    image = molmo_transform(image_pil).unsqueeze(0)
    with torch.no_grad():
        features = molmo7bd_model(image)
    return features.squeeze().numpy().flatten()

In [9]:
# Hand-crafted feature extraction (from your original code)
def extract_glcm_features(image):
    # image_uint8 = (image * 255).astype(np.uint8)
    glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    return np.array([contrast, dissimilarity])

def extract_wavelet_features(image):
    coeffs2 = pywt.dwt2(image, 'haar')
    LL, (LH, HL, HH) = coeffs2
    return np.concatenate([LL.flatten(), LH.flatten(), HL.flatten(), HH.flatten()])

def extract_hog_features(image):
    image = cv2.resize(image, (128, 128))
    features, _ = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return features

In [10]:
# Fixed feature size (adjust based on combined features)
FIXED_FEATURE_SIZE = 3000  # Adjust after testing actual sizes

# Preprocess and extract features for all splits
def preprocess_and_extract(model_name, extract_deep_fn, transform):
    feature_dataset = []
    label_dataset = []
    
    for split in ['train', 'valid', 'test']:
        split_dir = os.path.join(data_dir, split)
        for class_name in os.listdir(split_dir):
            class_dir = os.path.join(split_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if image is None:
                    continue
                
                image_pil = Image.open(img_path).convert('RGB')
                deep_features = extract_deep_fn(image_pil)
                glcm_features = extract_glcm_features(image)
                wavelet_features = extract_wavelet_features(image)
                hog_features = extract_hog_features(image)
                
                all_features = np.concatenate([deep_features, glcm_features, wavelet_features, hog_features])
                if all_features.shape[0] < FIXED_FEATURE_SIZE:
                    all_features = np.pad(all_features, (0, FIXED_FEATURE_SIZE - all_features.shape[0]), mode='constant')
                else:
                    all_features = all_features[:FIXED_FEATURE_SIZE]
                
                feature_dataset.append(all_features)
                label_dataset.append(class_name)
    
    features = np.array(feature_dataset, dtype=np.float32)
    labels = np.array(label_dataset)
    
    # Save combined features and labels
    np.save(os.path.join(processed_dir, f"{model_name}_featuresc.npy"), features)
    np.save(os.path.join(processed_dir, f"{model_name}_labelsc.npy"), labels)
    print(f"Combined features and labels saved for {model_name}")

In [11]:
# Extract features for all splits and models
models_dict = {
    "resnet50": (extract_resnet50_features, resnet_transform),
    "inceptionv3": (extract_inceptionv3_features, inception_transform),
    "efficientnetb0": (extract_efficientnetb0_features, efficient_transform),
    "densenet121": (extract_densenet121_features, densenet_transform),
    "molmo7bd": (extract_molmo7bd_features, molmo_transform)
}

In [12]:
# Extract features for all models
for model_name, (extract_deep_fn, transform) in models_dict.items():
    if not os.path.exists(data_dir):
        print(f"{data_dir} doesn't exist!")
        continue
    print(f"Extracting features for {model_name}...")
    preprocess_and_extract(model_name, extract_deep_fn, transform)

Extracting features for resnet50...
Combined features and labels saved for resnet50
Extracting features for inceptionv3...
Combined features and labels saved for inceptionv3
Extracting features for efficientnetb0...
Combined features and labels saved for efficientnetb0
Extracting features for densenet121...
Combined features and labels saved for densenet121
Extracting features for molmo7bd...
Combined features and labels saved for molmo7bd


In [13]:
# Train and evaluate classifier
def train_and_evaluate(model_name):
    all_features = np.load(os.path.join(processed_dir, f"{model_name}_featuresc.npy"))
    all_labels = np.load(os.path.join(processed_dir, f"{model_name}_labelsc.npy"))

    # Train/test split from combined data
    from sklearn.model_selection import train_test_split
    train_features, test_features, train_labels, test_labels = train_test_split(
        all_features, all_labels, test_size=0.2, random_state=42
    )

    le = LabelEncoder()
    train_labels_enc = le.fit_transform(train_labels)
    test_labels_enc = le.transform(test_labels)

    clf = SVC(kernel='linear', probability=True)
    clf.fit(train_features, train_labels_enc)
    
    test_preds = clf.predict(test_features)
    accuracy = accuracy_score(test_labels_enc, test_preds)
    precision = precision_score(test_labels_enc, test_preds, average='weighted')
    recall = recall_score(test_labels_enc, test_preds, average='weighted')
    f1 = f1_score(test_labels_enc, test_preds, average='weighted')
    
    return {"Model": model_name, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1-Score": f1}

In [14]:
# Evaluate all models
results = []
for model_name in models_dict.keys():
    result = train_and_evaluate(model_name)
    results.append(result)

# Generate comparison table
results_df = pd.DataFrame(results)
print("\nComparison Table:")
print(results_df.to_string(index=False))

# Save results
results_df.to_csv(os.path.join(processed_dir, "model_comparisonc.csv"), index=False)
print("Results saved to 'Processed_Data/model_comparisonc.csv'")


Comparison Table:
         Model  Accuracy  Precision  Recall  F1-Score
      resnet50      0.59   0.593986    0.59  0.584051
   inceptionv3      0.58   0.577996    0.58  0.573715
efficientnetb0      0.57   0.560212    0.57  0.562135
   densenet121      0.91   0.909698    0.91  0.909724
      molmo7bd      0.59   0.593986    0.59  0.584051
Results saved to 'Processed_Data/model_comparisonc.csv'
